In [40]:
import psycopg2 as pg
import pandas.io.sql as psql



In [41]:
connection = pg.connect(
    host="localhost",
    dbname="app",
    user="football", 
    password="football")

In [51]:
split_date = '2022-10-31'
train = psql.read_sql(f"SELECT * FROM data_for_statistics_preparation_total where fixture_timestamp_utc < '{split_date}'", connection)
test = psql.read_sql(f"SELECT f.*, o.odd, o.betting_type FROM data_for_statistics_preparation_total f left join public.module1_odd o on o.fixture_id = f.fixture_id and betting_type = 'over25' and bookmaker = 'Bet365' where f.fixture_timestamp_utc between '{split_date}' and '2023-03-10';", connection)
train.head()

,fixture_id,fixture_timestamp_utc,country,league_name,home_team_name,away_team_name,goals_home,goals_away,total,goals_last_3_home_matches,goals_last_3_away_matches,more_than_6_goals_last_3_home_matches,num_matches_greater_2_goals_last_3_home_matches,number_matches_atleast_one_team_not_scored_last_3_home_matches,more_than_6_goals_last_3_away_matches,num_matches_greater_2_goals_last_3_away_matches,num_goals_last_match,num_matches_greater_0_away_goals_last_3_away_matches,home_team_points,away_team_points
0,896231,2022-10-14 16:30:00+00:00,Austria,Regionalliga - Salzburg,Grödig,Pinzgau Saalfelden,1,2,3,7.0,10.0,True,1,1,True,3,3.0,3,7.5,8.0
1,731393,2021-12-03 19:00:00+00:00,Portugal,Primeira Liga,Portimonense,FC Porto,0,3,3,4.0,10.0,False,0,3,True,3,3.0,3,-1.0,9.0
2,879972,2022-10-23 13:30:00+00:00,Russia,Premier League,Spartak Moscow,Khimki,5,0,5,11.0,11.0,True,2,1,True,3,3.0,3,9.5,4.0
3,154018,2019-08-31 14:00:00+00:00,Russia,First League,Tekstilshchik,Fakel Voronezh,1,4,5,8.0,7.0,True,1,1,True,2,3.0,1,7.0,1.5
4,733511,2022-03-04 18:00:00+00:00,Germany,Regionalliga - Nordost,Auerbach,Tasmania Berlin,9,1,10,13.0,12.0,True,2,1,True,1,2.0,1,2.0,5.5


In [52]:
train["over25"] = [True if res >= 3 else False for res in train['total']]
actuals = [True if res >= 3 else False for res in test['total']]

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

import xgboost as xgb

train_features = [
    "goals_last_3_home_matches",
    "goals_last_3_away_matches",
    "more_than_6_goals_last_3_home_matches",
    "num_matches_greater_2_goals_last_3_home_matches",
    "number_matches_atleast_one_team_not_scored_last_3_home_matches",
    "more_than_6_goals_last_3_away_matches",
    "num_matches_greater_2_goals_last_3_away_matches",
    "num_goals_last_match",
    "num_matches_greater_0_away_goals_last_3_away_matches",
    "home_team_points",
    "away_team_points"    
]

categorical_features=[]

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ]
)
param_dict = {
    "max_delta_step": 1,
    "scale_pos_weight": 1
}

classifier = xgb.XGBClassifier(
    max_delta_step=param_dict["max_delta_step"],
    scale_pos_weight=param_dict["scale_pos_weight"],
    objective="binary:logistic"
)

pipeline = Pipeline(
    steps=[
        #('preprocessor', preprocessor),
        ('classifier', classifier)
    ]
)

y = train[["over25"]].values.ravel()
x = train[train_features]

pipeline.fit(x, y)

Pipeline(steps=[('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.300000012, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=1,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=0, reg_alpha=0,
                               reg_lambda=1, ...))])

In [54]:
from sklearn.metrics import confusion_matrix

preds = pipeline.predict(test[train_features])
predicted_probs  = pipeline.predict_proba(test[train_features])[:, 1]

tn, fp, fn, tp = confusion_matrix(actuals, preds).ravel()
print("Accuracy :", (tp + tn) / (tn + fp + fn + tp))
print("Recall :", tp/ (tn + fn))

Accuracy : 0.537534791252485
Recall : 0.24707042580345748


In [56]:
test["own_odd"] = 1/predicted_probs
filtered_df = test[test["odd"] > 0]
thres = 0.5
filtered_df = filtered_df[filtered_df["odd"] - filtered_df["own_odd"] > thres]

money = 0
for _, row in filtered_df.iterrows():
    if row.total > 2:
        money += row.odd - 1
    else:
        money -= 1

print("Number Games: ", filtered_df.shape[0])
money

Number Games:  859


-110.01000000000002

# System


In [47]:
test = psql.read_sql(f"""SELECT fixture.fixture_id, fixture.fixture_date_utc,to_char(fixture.fixture_timestamp_utc, 'HH24:MI:SS'::text) AS fixture_time_utc, league.country,
 league.league_name, 
    league.round,
    fixture.home_team_name,
    fixture.away_team_name,
    fixture.goals_away + fixture.goals_home as total,
    odds.odd
   FROM module1_over25 calc
     JOIN module1_fixture fixture ON fixture.fixture_id = calc.fixture_id
     LEFT JOIN module1_leagueseason league ON fixture.league_season_id = league.league_season_id
     LEFT JOIN currentodds odds ON odds.fixture_id::text = calc.fixture_id AND odds.betting_type = 'over25'::text AND odds.bookmaker = 'Bet365'::text
  WHERE fixture.fixture_date_utc  between '{split_date}' and '2023-03-10'
  ORDER BY fixture.fixture_timestamp_utc;""", connection)

In [48]:
test

,fixture_id,fixture_date_utc,fixture_time_utc,country,league_name,round,home_team_name,away_team_name,total,odd
0,968154,2022-10-31,00:00:00,USA,Major League Soccer,Regular Season - 1,Philadelphia Union,New York City FC,4.0,1.95
1,963077,2022-10-31,01:00:00,Nicaragua,Primera Division,Apertura - 1,Juventus Managua,UNAN Managua,3.0,1.70
2,884442,2022-10-31,17:00:00,Turkey,Süper Lig,Regular Season - 1,Sivasspor,Antalyaspor,2.0,1.95
3,833079,2022-10-31,18:00:00,Sweden,Ettan - Norra,Norra - 1,Vasalund,Täby,6.0,1.36
4,828593,2022-10-31,23:00:00,Chile,Primera División,Regular Season - 1,U. Catolica,A. Italiano,0.0,1.85
...,...,...,...,...,...,...,...,...,...,...
1197,904896,2023-03-10,18:00:00,Germany,Regionalliga - SudWest,Südwest - 1,Kickers Offenbach,SGV Freiberg,3.0,1.60
1198,896414,2023-03-10,18:00:00,Austria,Regionalliga - Mitte,Mitte - 1,Treibach,Weiz,3.0,1.53
1199,904889,2023-03-10,18:00:00,Germany,Regionalliga - SudWest,Südwest - 1,FC Astoria Walldorf,FSV Frankfurt,3.0,1.80
1200,896126,2023-03-10,18:30:00,Austria,Regionalliga - Tirol,Tirol - 1,Wörgl,Schwaz,3.0,NaN


In [49]:
filtered_df = test[test["odd"] > 0]

money = 0
for _, row in filtered_df.iterrows():
    if row.total > 2:
        money += row.odd - 1
    else:
        money -= 1

print("Number Games: ", filtered_df.shape[0])
money

Number Games:  1010


-97.66000000000004

In [50]:
import http.client
import json
from dot_env import load_dotenv

load_dotenv()

api_result = []

for fixture in test["fixture_id"]:
    conn = http.client.HTTPSConnection("v3.football.api-sports.io")

    headers = {
        'x-rapidapi-host': "v3.football.api-sports.io",
        'x-rapidapi-key': os.getenv('FOOTBALL_API_KEY')
        }

    conn.request("GET", f"/predictions?fixture={fixture}", headers=headers)

    res = conn.getresponse()
    data = res.read()
    res = json.loads(data.decode("utf-8"))
    pred = res["response"][0]["predictions"]["under_over"]
    api_result.append(pred)
    print(f"Fixture: {fixture}, Prediction: {pred}")
#print(data.decode("utf-8")[0]["response"])

ModuleNotFoundError: No module named 'dot_env'

In [ ]:
[True if res in ["+2.5", "+3.5", "+4.5"] else False for res in api_result]

In [ ]:
product_category = psql.read_sql_query('SELECT * FROM xgstatistics limit 100', connection)
product_category.head()

/opt/conda/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,fixture_id,home_goals_for_sma500,home_goals_against_sma500
0,172691,NaN,NaN
1,172693,2.0,2.0
2,172692,1.0,1.5
3,172695,1.0,1.0
4,172696,0.8,1.3


In [ ]:
import mlflow